it is just a proof of concept, assume the lakehouse has a schema, if you have a space in your workspace or lakehouse, use id instead of the name

In [ ]:
# one off operation
!pip install  duckdb --upgrade
#!pip install  altair --upgrade
#!pip install azure-identity
# in order to use the updated duckdb package you need to restart python notebook
# https://learn.microsoft.com/en-us/fabric/data-engineering/using-python-experience-on-notebook#library-management
notebookutils.session.restartPython()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 48.1 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: duckdb
    Found existing installation: duckdb 1.2.2
    Uninstalling duckdb-1.2.2:
      Successfully uninstalled duckdb-1.2.2
sys.exit called with value 0. The interpreter will be restarted.


In [28]:
from   psutil import *
import time
import duckdb
import altair as alt

In [24]:
def get_token():
    try:
      token = notebookutils.credentials.getToken('storage')
      return token
    except:
        from azure.identity import InteractiveBrowserCredential
        token = InteractiveBrowserCredential().get_token("https://storage.azure.com/.default")
        return token.token

In [31]:
def duckdb_attach_lakehouse(token,workspace,LH):
    con = duckdb.connect(f'temp_{time.time_ns()}.duckdb')
    con.sql('SET enable_object_cache=true')
    con.sql(f""" CREATE or replace SECRET onelake ( TYPE AZURE, PROVIDER ACCESS_TOKEN, ACCESS_TOKEN '{token}')   """)
    
    sql_schema     = set()
    sql_statements = set()
    
    con.sql(f""" SELECT  * FROM glob ("abfss://{workspace}@onelake.dfs.fabric.microsoft.com/{LH}.Lakehouse/Tables/dbo/*") """).df()['file'].tolist()
    list_tables = con.sql(f""" SELECT  distinct(split_part(file, '_delta_log', 1)) as tables FROM glob ("abfss://{workspace}@onelake.dfs.fabric.microsoft.com/{LH}.Lakehouse/Tables/*/*/_delta_log/*.json") """).df()['tables'].tolist()
    for table_path in list_tables:
            parts = table_path.strip("/").split("/")
            schema = parts[-2]
            table = parts[-1]
            sql_schema.add(f"CREATE SCHEMA IF NOT EXISTS {schema};")
            sql_statements.add(f"""CREATE OR REPLACE view {schema}.{table} AS SELECT * 
                                FROM delta_scan('abfss://{workspace}@onelake.dfs.fabric.microsoft.com/{LH}.Lakehouse/Tables/{schema}/{table}');""")
    con.sql(" ".join(sql_schema))
    con.sql(" ".join(sql_statements))
    con.sql("SHOW ALL TABLES").show(max_width=150)
    con.sql('CHECKPOINT')
    return con

In [4]:
core = cpu_count()
vCPU = str(core) + " vCPU"
mem=round(virtual_memory().total/(1024 * 1024 * 1024),0)
print(vCPU +' Memory:'+ str(mem))

8 vCPU Memory:32.0


In [25]:
token = get_token()

In [32]:
%%time
con = duckdb_attach_lakehouse(token,"taxi","NYT")

┌──────────────────────┬─────────┬─────────┬──────────────────────┬──────────────────────────────────────────────────────────────────────┬───────────┐
│       database       │ schema  │  name   │     column_names     │                             column_types                             │ temporary │
│       varchar        │ varchar │ varchar │      varchar[]       │                              varchar[]                               │  boolean  │
├──────────────────────┼─────────┼─────────┼──────────────────────┼──────────────────────────────────────────────────────────────────────┼───────────┤
│ temp_1734354510296…  │ ny      │ taxi    │ [trip_distance, st…  │ [DOUBLE, VARCHAR, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, DOUBLE, …  │ false     │
└──────────────────────┴─────────┴─────────┴──────────────────────┴──────────────────────────────────────────────────────────────────────┴───────────┘

CPU times: total: 2.34 s
Wall time: 13 s


In [33]:
con.sql(""" select count(*) from ny.taxi """) 

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│   1398950925 │
└──────────────┘

In [34]:
con.sql(""" 
        -- the summary table will be saved in the local SSD,
        create or replace table summary as 
        SELECT
        date ,
        SUM(fare_amount) AS totalfares ,
        AVG(fare_amount) AS avgfares
        from ny.taxi 
        --where year > 2020
        group by date
        order by date desc
        """)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [35]:
agg=con.sql(' from summary').df()

In [36]:
brush = alt.selection_interval()
details = alt.Chart(agg).mark_bar().encode(alt.X('date:T'),  alt.Y('totalfares:Q'), tooltip=[alt.Tooltip('date:T',format='%Y-%m-%d %H'),'totalfares:Q']
).properties( width=1400,  height=400 ).add_params( brush)

summary = alt.Chart(agg).mark_square().encode( alt.X('date:T'), alt.Y('avgfares:Q'), tooltip=['avgfares:Q'] ).properties(  width=1400,  height=400).transform_filter( brush)
details & summary

alt.VConcatChart(...)